<div style="text-align: center; margin-left: 0em; font-weight: bold; font-size: 20px; font-family: TimesNewRoman;">
    TIME SERIES DATA PROCESSING - Main Notebook
</div>

<div style="text-align: left; margin-left: 0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
Each part of the following script was used to proccess the raw data for the Availability Factor Time Series Raw Data for all the european countries of the Dispa-SET_Unleash project.
<br>
Read explanation text cells to follow and understand all the process until final results were got stept by step.
</div>

<div style="text-align: left; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;">
    1. Notebook Set Up
</div>

<div style="text-align: left; margin-left: 0.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
    Importing needed libraries
</div>

In [35]:
import pandas as pd
import re
import csv
import os
import requests
from urllib.parse import urlparse, parse_qs
import datetime

<div style="text-align: left; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;">
    2. Dispa-SET_Unleash Folder Path
</div>

<div style="text-align: left; margin-left: 0.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
    Determinning dynamically the zone_folder_path based on the location of the "Dispa-SET_Unleash" folder relative to the current working directory. 
<br>
    If the "Dispa-SET_Unleash" folder is copied to a different machine or location, the dispaSET_unleash_folder_path variable will automatically adjust accordingly.
</div>

In [36]:
# Get the current working directory
current_directory = os.getcwd()

# Navigate to the parent directory of "Dispa-SET_Unleash"
dispaSET_unleash_parent_directory = os.path.dirname(current_directory)

# Get the path to the "Dispa-SET_Unleash" folder
dispaSET_unleash_folder_path = os.path.dirname(dispaSET_unleash_parent_directory)

# Construct the dispaSET_unleash_folder_name variable
dispaSET_unleash_folder_name = os.path.basename(dispaSET_unleash_folder_path)

print("dispaSET_unleash_folder_name:", dispaSET_unleash_folder_name)
print("dispaSET_unleash_folder_path:", dispaSET_unleash_folder_path)

dispaSET_unleash_folder_name: Dispa-SET_Unleash
dispaSET_unleash_folder_path: /home/ray/Dispa-SET_Unleash


<div style="text-align: left; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;">
    3. Usefull Variable Definition
</div>

<div style="text-align: left; margin-left: 0.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
    Entering a value to all the variables which content are going to be used in some of the next stages of this script. 
<br>
    Indicate the year of all data is referring to in the variable data_year.
<br>
    The universal_standar_time variable is going to be used to download all the time series data in this horary zone. Additionally as each european country belongs a particular time sector the corresponding time series data related to its time sector are going to be downloaded as well but in a different file.
</div>

In [37]:
# Year to which data refers to:
data_year = '2023'

# Universal standad time:
universal_standard_time = 'UTC'

# Western European Time:
western_european_time = 'WET_WEST'

# Central European Time:
central_european_time = 'CET_CEST'

# Eastern European Time:
eastern_european_time = 'EET_EST'

<div style="text-align: left; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;">
4. European Standard Time
</div>

<div style="text-align: left; margin-left: 0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
Each of the Europeand Countries Modeled in Dispa-SET are in a particular european standar time zone. So in order to get the appropiated time series data frame, the correspondign european time zone is going to be identificated for each country.
<br>
Additionally all the time series is also going to be downloaded under the UTC <em>(The World’s Time Standard)</em>.
<br>
This is going to be done for each country as well.
    <br>
All this features are going to be saved in a csv file called Standar_Time_Data.csv where all additional characters to the download process are going to be written
</div>

In [38]:
# Additional string to be appended
additional_path = "/RawData/AvailabiltyFactors/"

# Construct the power_plants_raw_data_folder_path variable
standard_time_data_folder_path = dispaSET_unleash_folder_path + additional_path
print("standard_time_data_folder_path:", standard_time_data_folder_path)

standard_time_data_folder_path: /home/ray/Dispa-SET_Unleash/RawData/AvailabiltyFactors/


In [39]:
# Define lists of countries and standard times
countries = [
    "Austria", "Belgium", "Bulgaria", "Switzerland", "Cyprus", "Czech Republic",
    "Germany", "Denmark", "Estonia", "Greece", "Spain", "Finland", "France",
    "Croatia", "Hungary", "Ireland", "Italy", "Lithuania", "Luxembourg", "Latvia",
    "Malta", "Netherlands", "Norway", "Poland", "Portugal", "Romania", "Sweden",
    "Slovenia", "Slovakia", "United Kingdom"
]

dispaSET_codes = ["AT", "BE", "BG", "CH", "CY", "CZ", "DE", "DK", "EE", "EL", "ES", "FI", "FR", "HR", "HU", 
                  "IE", "IT", "LT", "LU", "LV", "MT", "NL", "NO", "PL", "PT", "RO", "SE", "SI", "SK", "UK"
]

standard_times_url_country_base = [
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|CET|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|CET|DAYTIMERANGE&area.values=CTY|10YAT-APG------L!CTY|10YAT-APG------L&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|CET|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|CET|DAYTIMERANGE&area.values=CTY|10YBE----------2!CTY|10YBE----------2&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|EET|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|EET|DAYTIMERANGE&area.values=CTY|10YCA-BULGARIA-R!CTY|10YCA-BULGARIA-R&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=EET_EEST&dateTime.timezone_input=EET+(UTC+2)+/+EEST+(UTC+3)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|CET|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|CET|DAYTIMERANGE&area.values=CTY|10YCH-SWISSGRIDZ!CTY|10YCH-SWISSGRIDZ&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|EET|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|EET|DAYTIMERANGE&area.values=CTY|10YCY-1001A0003J!CTY|10YCY-1001A0003J&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=EET_EEST&dateTime.timezone_input=EET+(UTC+2)+/+EEST+(UTC+3)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|CET|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|CET|DAYTIMERANGE&area.values=CTY|10YCZ-CEPS-----N!CTY|10YCZ-CEPS-----N&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|CET|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|CET|DAYTIMERANGE&area.values=CTY|10Y1001A1001A83F!CTY|10Y1001A1001A83F&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|CET|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|CET|DAYTIMERANGE&area.values=CTY|10Y1001A1001A65H!CTY|10Y1001A1001A65H&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|EET|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|EET|DAYTIMERANGE&area.values=CTY|10Y1001A1001A39I!CTY|10Y1001A1001A39I&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=EET_EEST&dateTime.timezone_input=EET+(UTC+2)+/+EEST+(UTC+3)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|EET|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|EET|DAYTIMERANGE&area.values=CTY|10YGR-HTSO-----Y!CTY|10YGR-HTSO-----Y&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=EET_EEST&dateTime.timezone_input=EET+(UTC+2)+/+EEST+(UTC+3)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|CET|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|CET|DAYTIMERANGE&area.values=CTY|10YES-REE------0!CTY|10YES-REE------0&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|EET|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|EET|DAYTIMERANGE&area.values=CTY|10YFI-1--------U!CTY|10YFI-1--------U&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=EET_EEST&dateTime.timezone_input=EET+(UTC+2)+/+EEST+(UTC+3)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|CET|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|CET|DAYTIMERANGE&area.values=CTY|10YFR-RTE------C!CTY|10YFR-RTE------C&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|CET|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|CET|DAYTIMERANGE&area.values=CTY|10YHR-HEP------M!CTY|10YHR-HEP------M&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|CET|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|CET|DAYTIMERANGE&area.values=CTY|10YHU-MAVIR----U!CTY|10YHU-MAVIR----U&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|CET|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|CET|DAYTIMERANGE&area.values=CTY|10YIE-1001A00010!CTY|10YIE-1001A00010&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|CET|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|CET|DAYTIMERANGE&area.values=CTY|10YIT-GRTN-----B!CTY|10YIT-GRTN-----B&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|EET|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|EET|DAYTIMERANGE&area.values=CTY|10YLT-1001A0008Q!CTY|10YLT-1001A0008Q&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=EET_EEST&dateTime.timezone_input=EET+(UTC+2)+/+EEST+(UTC+3)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|CET|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|CET|DAYTIMERANGE&area.values=CTY|10YLU-CEGEDEL-NQ!CTY|10YLU-CEGEDEL-NQ&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|EET|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|EET|DAYTIMERANGE&area.values=CTY|10YLV-1001A00074!CTY|10YLV-1001A00074&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=EET_EEST&dateTime.timezone_input=EET+(UTC+2)+/+EEST+(UTC+3)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|CET|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|CET|DAYTIMERANGE&area.values=CTY|10Y1001A1001A93C!CTY|10Y1001A1001A93C&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|CET|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|CET|DAYTIMERANGE&area.values=CTY|10YNL----------L!CTY|10YNL----------L&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|CET|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|CET|DAYTIMERANGE&area.values=CTY|10YNO-0--------C!CTY|10YNO-0--------C&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|CET|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|CET|DAYTIMERANGE&area.values=CTY|10YPL-AREA-----S!CTY|10YPL-AREA-----S&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|WET|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|WET|DAYTIMERANGE&area.values=CTY|10YPT-REN------W!CTY|10YPT-REN------W&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=WET_WEST&dateTime.timezone_input=WET+(UTC)+/+WEST+(UTC+1)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|EET|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|EET|DAYTIMERANGE&area.values=CTY|10YRO-TEL------P!CTY|10YRO-TEL------P&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=EET_EEST&dateTime.timezone_input=EET+(UTC+2)+/+EEST+(UTC+3)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|CET|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|CET|DAYTIMERANGE&area.values=CTY|10YSE-1--------K!CTY|10YSE-1--------K&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|CET|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|CET|DAYTIMERANGE&area.values=CTY|10YSI-ELES-----O!CTY|10YSI-ELES-----O&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|CET|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|CET|DAYTIMERANGE&area.values=CTY|10YSK-SEPS-----K!CTY|10YSK-SEPS-----K&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|WET|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|WET|DAYTIMERANGE&area.values=CTY|10Y1001A1001A92E!CTY|10Y1001A1001A92E&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=WET_WEST&dateTime.timezone_input=WET+(UTC)+/+WEST+(UTC+1)"
]

# Create DataFrame
df = pd.DataFrame({'Country': countries, 'Dispa-SET_Code': dispaSET_codes, 'Full_Standard_Time_Url_Country_Base': standard_times_url_country_base})

# Add new empty column
df['Time_Zone_Country_Code'] = ''

df

standard_time_data_file_name = 'Standard_Time_Data.csv'

# Construct the full file path
standard_time_data_file_path = os.path.join(standard_time_data_folder_path, standard_time_data_file_name)

# Create the CSV file with the specified name
with open(standard_time_data_file_path, 'w') as f:
    # Optional: Write a header if needed
    # f.write("header1,header2,header3\n")

# Save DataFrame to the CSV file
    df.to_csv(standard_time_data_file_path, index=False)

In [40]:
# Load the CSV file into a pandas DataFrame
data = pd.read_csv(standard_time_data_file_path)

# Function to extract time zone country code from the URL
def extract_time_zone_country_code(url):
    start_index = url.find("&dateTime.timezone_input=") + len("&dateTime.timezone_input=")
    if start_index != -1:  # Check if delimiter is found
        return url[start_index:]  # Extract from start_index till the end
    else:
        return None

# Apply the function to extract time zone country code for each row
data['Time_Zone_Country_Code'] = data['Full_Standard_Time_Url_Country_Base'].apply(extract_time_zone_country_code)

# Write the DataFrame back to the CSV file with the new column included
data.to_csv(standard_time_data_file_path, index=False)

data

,Country,Dispa-SET_Code,Full_Standard_Time_Url_Country_Base,Time_Zone_Country_Code
0,Austria,AT,https://transparency.entsoe.eu/generation/r2/a...,CET+(UTC+1)+/+CEST+(UTC+2)
1,Belgium,BE,https://transparency.entsoe.eu/generation/r2/a...,CET+(UTC+1)+/+CEST+(UTC+2)
2,Bulgaria,BG,https://transparency.entsoe.eu/generation/r2/a...,EET+(UTC+2)+/+EEST+(UTC+3)
3,Switzerland,CH,https://transparency.entsoe.eu/generation/r2/a...,CET+(UTC+1)+/+CEST+(UTC+2)
4,Cyprus,CY,https://transparency.entsoe.eu/generation/r2/a...,EET+(UTC+2)+/+EEST+(UTC+3)
5,Czech Republic,CZ,https://transparency.entsoe.eu/generation/r2/a...,CET+(UTC+1)+/+CEST+(UTC+2)
6,Germany,DE,https://transparency.entsoe.eu/generation/r2/a...,CET+(UTC+1)+/+CEST+(UTC+2)
7,Denmark,DK,https://transparency.entsoe.eu/generation/r2/a...,CET+(UTC+1)+/+CEST+(UTC+2)
8,Estonia,EE,https://transparency.entsoe.eu/generation/r2/a...,EET+(UTC+2)+/+EEST+(UTC+3)
9,Greece,EL,https://transparency.entsoe.eu/generation/r2/a...,EET+(UTC+2)+/+EEST+(UTC+3)


In [41]:
# Open the existing CSV file in read mode
with open(standard_time_data_file_path, 'r') as csvfile:
    # Read existing data
    reader = csv.reader(csvfile)
    rows = list(reader)

# Add new columns to the header row
header_row = rows[0]
header_row.extend(['Data_Year', 'Universal_Standard_Time', 'Western_European_Time', 'Central_European_Time', 'Eastern_European_Time'])

# Add data to each row
for row in rows[1:]:
    row.extend([data_year, universal_standard_time, western_european_time, central_european_time, eastern_european_time])


# Write back to the CSV file
with open(standard_time_data_file_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(rows)

print("Columns added successfully to the CSV file at:", standard_time_data_file_path)

Columns added successfully to the CSV file at: /home/ray/Dispa-SET_Unleash/RawData/AvailabiltyFactors/Standard_Time_Data.csv


<div style="text-align: right; margin-left: 3.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman;">
    Tracking Variables. 
    <br>
    <div style="text-align: right; margin-left: 1.50em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman;">
    This cells are just to confirm all the file names, file paths and other information related to the data being processed.
    <br>
  Also are used to ensure the inputs for next cells in order to avoid to re-enter the same information each time.
</div>

In [42]:
print (f"dispaSET_unleash_folder_name:                              {dispaSET_unleash_folder_name}")
print (f"dispaSET_unleash_folder_path:                              {dispaSET_unleash_folder_path}")
print (f"data_year:                                                 {data_year}")
print (f"universal_standard_time:                                   {universal_standard_time}")
print (f"western_european_time:                                     {western_european_time}")
print (f"central_european_time:                                     {central_european_time}")
print (f"eastern_european_time:                                     {eastern_european_time}")
print (f"standard_time_data_folder_path:                            {standard_time_data_folder_path}")
print (f"standard_time_data_file_name:                              {standard_time_data_file_name}")
print (f"standard_time_data_file_path:                              {standard_time_data_file_path}")

dispaSET_unleash_folder_name:                              Dispa-SET_Unleash
dispaSET_unleash_folder_path:                              /home/ray/Dispa-SET_Unleash
data_year:                                                 2023
universal_standard_time:                                   UTC
western_european_time:                                     WET_WEST
central_european_time:                                     CET_CEST
eastern_european_time:                                     EET_EST
standard_time_data_folder_path:                            /home/ray/Dispa-SET_Unleash/RawData/AvailabiltyFactors/
standard_time_data_file_name:                              Standard_Time_Data.csv
standard_time_data_file_path:                              /home/ray/Dispa-SET_Unleash/RawData/AvailabiltyFactors/Standard_Time_Data.csv


<div style="text-align: left; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;">
5. Main Data Source
</div>

<div style="text-align: left; margin-left: 0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
All the data to be processed is going to be downloaded from one main source:
<br>
<em><strong>ENTSOE Transparency Platform:</strong></em> Which the main url link is the following
<div style="text-align: left; margin-left: 3.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
https://transparency.entsoe.eu/dashboard/show
</div>

<div style="text-align: left; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman;">
5.1. Url Donwload Link Base
</div>

<div style="text-align: left; margin-left: 0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
All the downloaded data use a base url structure as download link.
<br>
The URL base is lightly modified to get the correponding for each hour and day of the whole year specified in the variable data_year.
     <br>
Additionally and as the ENTSO E web page provides time series in four different Time zones, all the data for aech country is going to be downloaded in the four horary usages belong present in the Dispa-SET countries, i.e. Universal Standat Time (UTC), Western_European_Time (WET / WEST), Central_European_Time (CET / CEST) and Eastern_European_Time (EET / EEST) 
    <br>
Each URL_country_base link is added to the csv file standard_time_data_file_path.
    <br>
The ENTSOE web portal do not provide the corresponding information for Malta. This country is going to be avoided in the tables of the next process. However, other sources are going to be looked for and their corresponding dowload and formating scrips are going to be developed later.
</div>

In [43]:
# List of URLs
utc_urls = [
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YAT-APG------L!CTY|10YAT-APG------L&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YBE----------2!CTY|10YBE----------2&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YCA-BULGARIA-R!CTY|10YCA-BULGARIA-R&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YCH-SWISSGRIDZ!CTY|10YCH-SWISSGRIDZ&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YCY-1001A0003J!CTY|10YCY-1001A0003J&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YCZ-CEPS-----N!CTY|10YCZ-CEPS-----N&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10Y1001A1001A83F!CTY|10Y1001A1001A83F&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10Y1001A1001A65H!CTY|10Y1001A1001A65H&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10Y1001A1001A83F!CTY|10Y1001A1001A83F&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YGR-HTSO-----Y!CTY|10YGR-HTSO-----Y&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YES-REE------0!CTY|10YES-REE------0&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YFI-1--------U!CTY|10YFI-1--------U&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YFR-RTE------C!CTY|10YFR-RTE------C&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YHR-HEP------M!CTY|10YHR-HEP------M&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YHU-MAVIR----U!CTY|10YHU-MAVIR----U&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YIE-1001A00010!CTY|10YIE-1001A00010&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YIT-GRTN-----B!CTY|10YIT-GRTN-----B&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YLT-1001A0008Q!CTY|10YLT-1001A0008Q&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YLU-CEGEDEL-NQ!CTY|10YLU-CEGEDEL-NQ&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YLV-1001A00074!CTY|10YLV-1001A00074&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10Y1001A1001A93C!CTY|10Y1001A1001A93C&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YNL----------L!CTY|10YNL----------L&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YNO-0--------C!CTY|10YNO-0--------C&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YPL-AREA-----S!CTY|10YPL-AREA-----S&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YPL-AREA-----S!CTY|10YPL-AREA-----S&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YRO-TEL------P!CTY|10YRO-TEL------P&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YSE-1--------K!CTY|10YSE-1--------K&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YSI-ELES-----O!CTY|10YSI-ELES-----O&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YSK-SEPS-----K!CTY|10YSK-SEPS-----K&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10Y1001A1001A92E!CTY|10Y1001A1001A92E&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=UTC&dateTime.timezone_input=UTC"
]

cet_cest_urls = [
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|CET|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|CET|DAYTIMERANGE&area.values=CTY|10YAT-APG------L!CTY|10YAT-APG------L&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|CET|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|CET|DAYTIMERANGE&area.values=CTY|10YBE----------2!CTY|10YBE----------2&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YCA-BULGARIA-R!CTY|10YCA-BULGARIA-R&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YCH-SWISSGRIDZ!CTY|10YCH-SWISSGRIDZ&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YCY-1001A0003J!CTY|10YCY-1001A0003J&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YCZ-CEPS-----N!CTY|10YCZ-CEPS-----N&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10Y1001A1001A83F!CTY|10Y1001A1001A83F&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10Y1001A1001A65H!CTY|10Y1001A1001A65H&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10Y1001A1001A83F!CTY|10Y1001A1001A83F&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YGR-HTSO-----Y!CTY|10YGR-HTSO-----Y&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YES-REE------0!CTY|10YES-REE------0&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YFI-1--------U!CTY|10YFI-1--------U&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YFR-RTE------C!CTY|10YFR-RTE------C&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YHR-HEP------M!CTY|10YHR-HEP------M&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YHU-MAVIR----U!CTY|10YHU-MAVIR----U&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YIE-1001A00010!CTY|10YIE-1001A00010&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YIT-GRTN-----B!CTY|10YIT-GRTN-----B&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YLT-1001A0008Q!CTY|10YLT-1001A0008Q&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YLU-CEGEDEL-NQ!CTY|10YLU-CEGEDEL-NQ&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YLV-1001A00074!CTY|10YLV-1001A00074&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10Y1001A1001A93C!CTY|10Y1001A1001A93C&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YNL----------L!CTY|10YNL----------L&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YNO-0--------C!CTY|10YNO-0--------C&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YPL-AREA-----S!CTY|10YPL-AREA-----S&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YPL-AREA-----S!CTY|10YPL-AREA-----S&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YRO-TEL------P!CTY|10YRO-TEL------P&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YSE-1--------K!CTY|10YSE-1--------K&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YSI-ELES-----O!CTY|10YSI-ELES-----O&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YSK-SEPS-----K!CTY|10YSK-SEPS-----K&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10Y1001A1001A92E!CTY|10Y1001A1001A92E&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)"
]

wet_west_urls = [
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|WET|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|WET|DAYTIMERANGE&area.values=CTY|10YAT-APG------L!CTY|10YAT-APG------L&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=WET_WEST&dateTime.timezone_input=WET+(UTC)+/+WEST+(UTC+1)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YBE----------2!CTY|10YBE----------2&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=WET_WEST&dateTime.timezone_input=WET+(UTC)+/+WEST+(UTC+1)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YCA-BULGARIA-R!CTY|10YCA-BULGARIA-R&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=WET_WEST&dateTime.timezone_input=WET+(UTC)+/+WEST+(UTC+1)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YCH-SWISSGRIDZ!CTY|10YCH-SWISSGRIDZ&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=WET_WEST&dateTime.timezone_input=WET+(UTC)+/+WEST+(UTC+1)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YCY-1001A0003J!CTY|10YCY-1001A0003J&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=WET_WEST&dateTime.timezone_input=WET+(UTC)+/+WEST+(UTC+1)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YCZ-CEPS-----N!CTY|10YCZ-CEPS-----N&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=WET_WEST&dateTime.timezone_input=WET+(UTC)+/+WEST+(UTC+1)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10Y1001A1001A83F!CTY|10Y1001A1001A83F&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=WET_WEST&dateTime.timezone_input=WET+(UTC)+/+WEST+(UTC+1)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10Y1001A1001A65H!CTY|10Y1001A1001A65H&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=WET_WEST&dateTime.timezone_input=WET+(UTC)+/+WEST+(UTC+1)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10Y1001A1001A83F!CTY|10Y1001A1001A83F&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=WET_WEST&dateTime.timezone_input=WET+(UTC)+/+WEST+(UTC+1)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YGR-HTSO-----Y!CTY|10YGR-HTSO-----Y&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=WET_WEST&dateTime.timezone_input=WET+(UTC)+/+WEST+(UTC+1)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YES-REE------0!CTY|10YES-REE------0&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=WET_WEST&dateTime.timezone_input=WET+(UTC)+/+WEST+(UTC+1)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YFI-1--------U!CTY|10YFI-1--------U&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=WET_WEST&dateTime.timezone_input=WET+(UTC)+/+WEST+(UTC+1)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YFR-RTE------C!CTY|10YFR-RTE------C&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=WET_WEST&dateTime.timezone_input=WET+(UTC)+/+WEST+(UTC+1)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YHR-HEP------M!CTY|10YHR-HEP------M&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=WET_WEST&dateTime.timezone_input=WET+(UTC)+/+WEST+(UTC+1)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YHU-MAVIR----U!CTY|10YHU-MAVIR----U&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=WET_WEST&dateTime.timezone_input=WET+(UTC)+/+WEST+(UTC+1)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YIE-1001A00010!CTY|10YIE-1001A00010&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=WET_WEST&dateTime.timezone_input=WET+(UTC)+/+WEST+(UTC+1)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YIT-GRTN-----B!CTY|10YIT-GRTN-----B&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=WET_WEST&dateTime.timezone_input=WET+(UTC)+/+WEST+(UTC+1)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YLT-1001A0008Q!CTY|10YLT-1001A0008Q&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=WET_WEST&dateTime.timezone_input=WET+(UTC)+/+WEST+(UTC+1)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YLU-CEGEDEL-NQ!CTY|10YLU-CEGEDEL-NQ&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=WET_WEST&dateTime.timezone_input=WET+(UTC)+/+WEST+(UTC+1)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YLV-1001A00074!CTY|10YLV-1001A00074&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=WET_WEST&dateTime.timezone_input=WET+(UTC)+/+WEST+(UTC+1)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10Y1001A1001A93C!CTY|10Y1001A1001A93C&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=WET_WEST&dateTime.timezone_input=WET+(UTC)+/+WEST+(UTC+1)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YNL----------L!CTY|10YNL----------L&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=WET_WEST&dateTime.timezone_input=WET+(UTC)+/+WEST+(UTC+1)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YNO-0--------C!CTY|10YNO-0--------C&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=WET_WEST&dateTime.timezone_input=WET+(UTC)+/+WEST+(UTC+1)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YPL-AREA-----S!CTY|10YPL-AREA-----S&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=WET_WEST&dateTime.timezone_input=WET+(UTC)+/+WEST+(UTC+1)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YPL-AREA-----S!CTY|10YPL-AREA-----S&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=WET_WEST&dateTime.timezone_input=WET+(UTC)+/+WEST+(UTC+1)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YRO-TEL------P!CTY|10YRO-TEL------P&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=WET_WEST&dateTime.timezone_input=WET+(UTC)+/+WEST+(UTC+1)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YSE-1--------K!CTY|10YSE-1--------K&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=WET_WEST&dateTime.timezone_input=WET+(UTC)+/+WEST+(UTC+1)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YSI-ELES-----O!CTY|10YSI-ELES-----O&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=WET_WEST&dateTime.timezone_input=WET+(UTC)+/+WEST+(UTC+1)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YSK-SEPS-----K!CTY|10YSK-SEPS-----K&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=WET_WEST&dateTime.timezone_input=WET+(UTC)+/+WEST+(UTC+1)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10Y1001A1001A92E!CTY|10Y1001A1001A92E&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=WET_WEST&dateTime.timezone_input=WET+(UTC)+/+WEST+(UTC+1)"
]

eet_eest_urls = [
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|EET|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|EET|DAYTIMERANGE&area.values=CTY|10YAT-APG------L!CTY|10YAT-APG------L&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=EET_EEST&dateTime.timezone_input=EET+(UTC+2)+/+EEST+(UTC+3)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YBE----------2!CTY|10YBE----------2&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=EET_EEST&dateTime.timezone_input=EET+(UTC+2)+/+EEST+(UTC+3)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YCA-BULGARIA-R!CTY|10YCA-BULGARIA-R&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=EET_EEST&dateTime.timezone_input=EET+(UTC+2)+/+EEST+(UTC+3)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YCH-SWISSGRIDZ!CTY|10YCH-SWISSGRIDZ&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=EET_EEST&dateTime.timezone_input=EET+(UTC+2)+/+EEST+(UTC+3)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YCY-1001A0003J!CTY|10YCY-1001A0003J&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=EET_EEST&dateTime.timezone_input=EET+(UTC+2)+/+EEST+(UTC+3)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YCZ-CEPS-----N!CTY|10YCZ-CEPS-----N&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=EET_EEST&dateTime.timezone_input=EET+(UTC+2)+/+EEST+(UTC+3)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10Y1001A1001A83F!CTY|10Y1001A1001A83F&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=EET_EEST&dateTime.timezone_input=EET+(UTC+2)+/+EEST+(UTC+3)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10Y1001A1001A65H!CTY|10Y1001A1001A65H&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=EET_EEST&dateTime.timezone_input=EET+(UTC+2)+/+EEST+(UTC+3)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10Y1001A1001A83F!CTY|10Y1001A1001A83F&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=EET_EEST&dateTime.timezone_input=EET+(UTC+2)+/+EEST+(UTC+3)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YGR-HTSO-----Y!CTY|10YGR-HTSO-----Y&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=EET_EEST&dateTime.timezone_input=EET+(UTC+2)+/+EEST+(UTC+3)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YES-REE------0!CTY|10YES-REE------0&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=EET_EEST&dateTime.timezone_input=EET+(UTC+2)+/+EEST+(UTC+3)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YFI-1--------U!CTY|10YFI-1--------U&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=EET_EEST&dateTime.timezone_input=EET+(UTC+2)+/+EEST+(UTC+3)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YFR-RTE------C!CTY|10YFR-RTE------C&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=EET_EEST&dateTime.timezone_input=EET+(UTC+2)+/+EEST+(UTC+3)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YHR-HEP------M!CTY|10YHR-HEP------M&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=EET_EEST&dateTime.timezone_input=EET+(UTC+2)+/+EEST+(UTC+3)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YHU-MAVIR----U!CTY|10YHU-MAVIR----U&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=EET_EEST&dateTime.timezone_input=EET+(UTC+2)+/+EEST+(UTC+3)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YIE-1001A00010!CTY|10YIE-1001A00010&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=EET_EEST&dateTime.timezone_input=EET+(UTC+2)+/+EEST+(UTC+3)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YIT-GRTN-----B!CTY|10YIT-GRTN-----B&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=EET_EEST&dateTime.timezone_input=EET+(UTC+2)+/+EEST+(UTC+3)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YLT-1001A0008Q!CTY|10YLT-1001A0008Q&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=EET_EEST&dateTime.timezone_input=EET+(UTC+2)+/+EEST+(UTC+3)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YLU-CEGEDEL-NQ!CTY|10YLU-CEGEDEL-NQ&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=EET_EEST&dateTime.timezone_input=EET+(UTC+2)+/+EEST+(UTC+3)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YLV-1001A00074!CTY|10YLV-1001A00074&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=EET_EEST&dateTime.timezone_input=EET+(UTC+2)+/+EEST+(UTC+3)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10Y1001A1001A93C!CTY|10Y1001A1001A93C&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=EET_EEST&dateTime.timezone_input=EET+(UTC+2)+/+EEST+(UTC+3)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YNL----------L!CTY|10YNL----------L&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=EET_EEST&dateTime.timezone_input=EET+(UTC+2)+/+EEST+(UTC+3)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YNO-0--------C!CTY|10YNO-0--------C&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=EET_EEST&dateTime.timezone_input=EET+(UTC+2)+/+EEST+(UTC+3)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YPL-AREA-----S!CTY|10YPL-AREA-----S&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=EET_EEST&dateTime.timezone_input=EET+(UTC+2)+/+EEST+(UTC+3)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YPL-AREA-----S!CTY|10YPL-AREA-----S&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=EET_EEST&dateTime.timezone_input=EET+(UTC+2)+/+EEST+(UTC+3)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YRO-TEL------P!CTY|10YRO-TEL------P&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=EET_EEST&dateTime.timezone_input=EET+(UTC+2)+/+EEST+(UTC+3)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YSE-1--------K!CTY|10YSE-1--------K&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=EET_EEST&dateTime.timezone_input=EET+(UTC+2)+/+EEST+(UTC+3)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YSI-ELES-----O!CTY|10YSI-ELES-----O&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=EET_EEST&dateTime.timezone_input=EET+(UTC+2)+/+EEST+(UTC+3)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10YSK-SEPS-----K!CTY|10YSK-SEPS-----K&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=EET_EEST&dateTime.timezone_input=EET+(UTC+2)+/+EEST+(UTC+3)",
    "https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&dateTime.endDateTime=01.01.2023+00:00|UTC|DAYTIMERANGE&area.values=CTY|10Y1001A1001A92E!CTY|10Y1001A1001A92E&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&showConsumption.values=SHOW_CONSUMPTION&dateTime.timezone=EET_EEST&dateTime.timezone_input=EET+(UTC+2)+/+EEST+(UTC+3)",
]

# Read the CSV file
df = pd.read_csv(standard_time_data_file_path)

# Add a new column "Url_Country_Base" and populate it with the URLs
df["UTC_Url_Country_Base"] = utc_urls
df["WET_WEST_Url_Country_Base"] = wet_west_urls
df["CET_CEST_Url_Country_Base"] = cet_cest_urls
df["EET_EEST_Url_Country_Base"] = eet_eest_urls


# Write the DataFrame back to the CSV file
df.to_csv(standard_time_data_file_path, index=False)

print("URLs have been added to the new column 'Url_Country_Base' in the CSV file:", standard_time_data_file_path)

URLs have been added to the new column 'Url_Country_Base' in the CSV file: /home/ray/Dispa-SET_Unleash/RawData/AvailabiltyFactors/Standard_Time_Data.csv


<div style="text-align: left; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman;">
5.2. Url Country Code
</div>

<div style="text-align: left; margin-left: 0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
Extracting the Country code from the base Url.
<br>
The country code for each country is stracted in a new column in the standard_time_data_file_path. 
<br>
This data is needed to perform the next download links recursively.
</div>

In [44]:
# Load the CSV file into a pandas DataFrame
data = pd.read_csv(standard_time_data_file_path)

# Function to extract country code from the URL
def extract_country_code(url):
    start_index = url.find("&area.values=") + len("&area.values=")
    end_index = url.find("&productionType")
    if start_index >= len("values=CTY|") and end_index > start_index:
        return url[start_index:end_index]
    else:
        return None

# Apply the function to extract country code for each row
data['Country_Code'] = data['UTC_Url_Country_Base'].apply(extract_country_code)

# Write the DataFrame back to the CSV file with the new column included
data.to_csv(standard_time_data_file_path, index=False)

data

,Country,Dispa-SET_Code,Full_Standard_Time_Url_Country_Base,Time_Zone_Country_Code,Data_Year,Universal_Standard_Time,Western_European_Time,Central_European_Time,Eastern_European_Time,UTC_Url_Country_Base,WET_WEST_Url_Country_Base,CET_CEST_Url_Country_Base,EET_EEST_Url_Country_Base,Country_Code
0,Austria,AT,https://transparency.entsoe.eu/generation/r2/a...,CET+(UTC+1)+/+CEST+(UTC+2),2023,UTC,WET_WEST,CET_CEST,EET_EST,https://transparency.entsoe.eu/generation/r2/a...,https://transparency.entsoe.eu/generation/r2/a...,https://transparency.entsoe.eu/generation/r2/a...,https://transparency.entsoe.eu/generation/r2/a...,CTY|10YAT-APG------L!CTY|10YAT-APG------L
1,Belgium,BE,https://transparency.entsoe.eu/generation/r2/a...,CET+(UTC+1)+/+CEST+(UTC+2),2023,UTC,WET_WEST,CET_CEST,EET_EST,https://transparency.entsoe.eu/generation/r2/a...,https://transparency.entsoe.eu/generation/r2/a...,https://transparency.entsoe.eu/generation/r2/a...,https://transparency.entsoe.eu/generation/r2/a...,CTY|10YBE----------2!CTY|10YBE----------2
2,Bulgaria,BG,https://transparency.entsoe.eu/generation/r2/a...,EET+(UTC+2)+/+EEST+(UTC+3),2023,UTC,WET_WEST,CET_CEST,EET_EST,https://transparency.entsoe.eu/generation/r2/a...,https://transparency.entsoe.eu/generation/r2/a...,https://transparency.entsoe.eu/generation/r2/a...,https://transparency.entsoe.eu/generation/r2/a...,CTY|10YCA-BULGARIA-R!CTY|10YCA-BULGARIA-R
3,Switzerland,CH,https://transparency.entsoe.eu/generation/r2/a...,CET+(UTC+1)+/+CEST+(UTC+2),2023,UTC,WET_WEST,CET_CEST,EET_EST,https://transparency.entsoe.eu/generation/r2/a...,https://transparency.entsoe.eu/generation/r2/a...,https://transparency.entsoe.eu/generation/r2/a...,https://transparency.entsoe.eu/generation/r2/a...,CTY|10YCH-SWISSGRIDZ!CTY|10YCH-SWISSGRIDZ
4,Cyprus,CY,https://transparency.entsoe.eu/generation/r2/a...,EET+(UTC+2)+/+EEST+(UTC+3),2023,UTC,WET_WEST,CET_CEST,EET_EST,https://transparency.entsoe.eu/generation/r2/a...,https://transparency.entsoe.eu/generation/r2/a...,https://transparency.entsoe.eu/generation/r2/a...,https://transparency.entsoe.eu/generation/r2/a...,CTY|10YCY-1001A0003J!CTY|10YCY-1001A0003J
5,Czech Republic,CZ,https://transparency.entsoe.eu/generation/r2/a...,CET+(UTC+1)+/+CEST+(UTC+2),2023,UTC,WET_WEST,CET_CEST,EET_EST,https://transparency.entsoe.eu/generation/r2/a...,https://transparency.entsoe.eu/generation/r2/a...,https://transparency.entsoe.eu/generation/r2/a...,https://transparency.entsoe.eu/generation/r2/a...,CTY|10YCZ-CEPS-----N!CTY|10YCZ-CEPS-----N
6,Germany,DE,https://transparency.entsoe.eu/generation/r2/a...,CET+(UTC+1)+/+CEST+(UTC+2),2023,UTC,WET_WEST,CET_CEST,EET_EST,https://transparency.entsoe.eu/generation/r2/a...,https://transparency.entsoe.eu/generation/r2/a...,https://transparency.entsoe.eu/generation/r2/a...,https://transparency.entsoe.eu/generation/r2/a...,CTY|10Y1001A1001A83F!CTY|10Y1001A1001A83F
7,Denmark,DK,https://transparency.entsoe.eu/generation/r2/a...,CET+(UTC+1)+/+CEST+(UTC+2),2023,UTC,WET_WEST,CET_CEST,EET_EST,https://transparency.entsoe.eu/generation/r2/a...,https://transparency.entsoe.eu/generation/r2/a...,https://transparency.entsoe.eu/generation/r2/a...,https://transparency.entsoe.eu/generation/r2/a...,CTY|10Y1001A1001A65H!CTY|10Y1001A1001A65H
8,Estonia,EE,https://transparency.entsoe.eu/generation/r2/a...,EET+(UTC+2)+/+EEST+(UTC+3),2023,UTC,WET_WEST,CET_CEST,EET_EST,https://transparency.entsoe.eu/generation/r2/a...,https://transparency.entsoe.eu/generation/r2/a...,https://transparency.entsoe.eu/generation/r2/a...,https://transparency.entsoe.eu/generation/r2/a...,CTY|10Y1001A1001A83F!CTY|10Y1001A1001A83F
9,Greece,EL,https://transparency.entsoe.eu/generation/r2/a...,EET+(UTC+2)+/+EEST+(UTC+3),2023,UTC,WET_WEST,CET_CEST,EET_EST,https://transparency.entsoe.eu/generation/r2/a...,https://transparency.entsoe.eu/generation/r2/a...,https://transparency.entsoe.eu/generation/r2/a...,https://transparency.entsoe.eu/generation/r2/a...,CTY|10YGR-HTSO-----Y!CTY|10YGR-HTSO-----Y


<div style="text-align: left; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman;">
5.3. Country / Zone Data  Folders
</div>

<div style="text-align: left; margin-left: 0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
Creating all the contening folders for each European country modelled in Dispa-SET.
<br>
This folders/directories are going to be used to storage all the time series data downloaded from ENTSO E web resource.
<br>
Additionally all the paths of the future created files are going to be written in the standard_time_data_file in order to use them for next download stages.
</div>

In [45]:
# Read the CSV file
df = pd.read_csv(standard_time_data_file_path)

# Create new column 'Zone_Folder_Path'
df['Zone_Folder_Path'] = standard_time_data_folder_path + df['Dispa-SET_Code']

# Ensure that the necessary folders exist
for folder_path in df['Zone_Folder_Path']:
    if not os.path.exists(folder_path):
        os.makedirs(folder_path, exist_ok=True)

# Create a new CSV file for each row and add 'UTC_Country_File_Path', 'WET_West_Country_File_Path', 'CET_Cest_Country_File_Path', and 'EET_Eest_Country_File_Path' to the DataFrame
utc_country_file_paths = []
wet_west_country_file_paths = []
cet_cest_country_file_paths = []
eet_eest_country_file_paths = []

for index, row in df.iterrows():
    # Construct file paths for 'UTC_Country_File_Path'
    utc_file_path = os.path.join(row['Zone_Folder_Path'], "UTC.csv")
    utc_country_file_paths.append(utc_file_path)
    
    # Create 'UTC_Country_File_Path' CSV file
    with open(utc_file_path, 'w') as f:
        f.write("This is the file for " + row['Dispa-SET_Code'] + " in UTC")  # Example content, you can adjust as needed

    # Construct file paths for 'WET_WEST_Country_File_Path'
    wet_west_file_path = os.path.join(row['Zone_Folder_Path'], "WET_WEST.csv")
    wet_west_country_file_paths.append(wet_west_file_path)
    
    # Create 'WET_WEST_Country_File_Path' CSV file
    with open(wet_west_file_path, 'w') as f:
        f.write("This is the file for " + row['Dispa-SET_Code'] + " in WET/WEST")  # Example content, you can adjust as needed

    # Construct file paths for 'CET_CEST_Country_File_Path'
    cet_cest_file_path = os.path.join(row['Zone_Folder_Path'], "CET_CEST.csv")
    cet_cest_country_file_paths.append(cet_cest_file_path)
    
    # Create 'CET_CEST_Country_File_Path' CSV file
    with open(cet_cest_file_path, 'w') as f:
        f.write("This is the file for " + row['Dispa-SET_Code'] + " in CET/CEST")  # Example content, you can adjust as needed

    # Construct file paths for 'EET_EEST_Country_File_Path'
    eet_eest_file_path = os.path.join(row['Zone_Folder_Path'], "EET_EEST.csv")
    eet_eest_country_file_paths.append(eet_eest_file_path)
    
    # Create 'EET_EEST_Country_File_Path' CSV file
    with open(eet_eest_file_path, 'w') as f:
        f.write("This is the file for " + row['Dispa-SET_Code'] + " in EET/EEST")  # Example content, you can adjust as needed

# Add 'UTC_Country_File_Path', 'WET_West_Country_File_Path', 'CET_Cest_Country_File_Path', and 'EET_Eest_Country_File_Path' columns to DataFrame
df['UTC_Country_File_Path'] = utc_country_file_paths
df['WET_WEST_Country_File_Path'] = wet_west_country_file_paths
df['CET_CEST_Country_File_Path'] = cet_cest_country_file_paths
df['EET_EEST_Country_File_Path'] = eet_eest_country_file_paths

# Write the updated DataFrame back to the original CSV file
df.to_csv(standard_time_data_file_path, index=False)

df

,Country,Dispa-SET_Code,Full_Standard_Time_Url_Country_Base,Time_Zone_Country_Code,Data_Year,Universal_Standard_Time,Western_European_Time,Central_European_Time,Eastern_European_Time,UTC_Url_Country_Base,WET_WEST_Url_Country_Base,CET_CEST_Url_Country_Base,EET_EEST_Url_Country_Base,Country_Code,Zone_Folder_Path,UTC_Country_File_Path,WET_WEST_Country_File_Path,CET_CEST_Country_File_Path,EET_EEST_Country_File_Path
0,Austria,AT,https://transparency.entsoe.eu/generation/r2/a...,CET+(UTC+1)+/+CEST+(UTC+2),2023,UTC,WET_WEST,CET_CEST,EET_EST,https://transparency.entsoe.eu/generation/r2/a...,https://transparency.entsoe.eu/generation/r2/a...,https://transparency.entsoe.eu/generation/r2/a...,https://transparency.entsoe.eu/generation/r2/a...,CTY|10YAT-APG------L!CTY|10YAT-APG------L,/home/ray/Dispa-SET_Unleash/RawData/Availabilt...,/home/ray/Dispa-SET_Unleash/RawData/Availabilt...,/home/ray/Dispa-SET_Unleash/RawData/Availabilt...,/home/ray/Dispa-SET_Unleash/RawData/Availabilt...,/home/ray/Dispa-SET_Unleash/RawData/Availabilt...
1,Belgium,BE,https://transparency.entsoe.eu/generation/r2/a...,CET+(UTC+1)+/+CEST+(UTC+2),2023,UTC,WET_WEST,CET_CEST,EET_EST,https://transparency.entsoe.eu/generation/r2/a...,https://transparency.entsoe.eu/generation/r2/a...,https://transparency.entsoe.eu/generation/r2/a...,https://transparency.entsoe.eu/generation/r2/a...,CTY|10YBE----------2!CTY|10YBE----------2,/home/ray/Dispa-SET_Unleash/RawData/Availabilt...,/home/ray/Dispa-SET_Unleash/RawData/Availabilt...,/home/ray/Dispa-SET_Unleash/RawData/Availabilt...,/home/ray/Dispa-SET_Unleash/RawData/Availabilt...,/home/ray/Dispa-SET_Unleash/RawData/Availabilt...
2,Bulgaria,BG,https://transparency.entsoe.eu/generation/r2/a...,EET+(UTC+2)+/+EEST+(UTC+3),2023,UTC,WET_WEST,CET_CEST,EET_EST,https://transparency.entsoe.eu/generation/r2/a...,https://transparency.entsoe.eu/generation/r2/a...,https://transparency.entsoe.eu/generation/r2/a...,https://transparency.entsoe.eu/generation/r2/a...,CTY|10YCA-BULGARIA-R!CTY|10YCA-BULGARIA-R,/home/ray/Dispa-SET_Unleash/RawData/Availabilt...,/home/ray/Dispa-SET_Unleash/RawData/Availabilt...,/home/ray/Dispa-SET_Unleash/RawData/Availabilt...,/home/ray/Dispa-SET_Unleash/RawData/Availabilt...,/home/ray/Dispa-SET_Unleash/RawData/Availabilt...
3,Switzerland,CH,https://transparency.entsoe.eu/generation/r2/a...,CET+(UTC+1)+/+CEST+(UTC+2),2023,UTC,WET_WEST,CET_CEST,EET_EST,https://transparency.entsoe.eu/generation/r2/a...,https://transparency.entsoe.eu/generation/r2/a...,https://transparency.entsoe.eu/generation/r2/a...,https://transparency.entsoe.eu/generation/r2/a...,CTY|10YCH-SWISSGRIDZ!CTY|10YCH-SWISSGRIDZ,/home/ray/Dispa-SET_Unleash/RawData/Availabilt...,/home/ray/Dispa-SET_Unleash/RawData/Availabilt...,/home/ray/Dispa-SET_Unleash/RawData/Availabilt...,/home/ray/Dispa-SET_Unleash/RawData/Availabilt...,/home/ray/Dispa-SET_Unleash/RawData/Availabilt...
4,Cyprus,CY,https://transparency.entsoe.eu/generation/r2/a...,EET+(UTC+2)+/+EEST+(UTC+3),2023,UTC,WET_WEST,CET_CEST,EET_EST,https://transparency.entsoe.eu/generation/r2/a...,https://transparency.entsoe.eu/generation/r2/a...,https://transparency.entsoe.eu/generation/r2/a...,https://transparency.entsoe.eu/generation/r2/a...,CTY|10YCY-1001A0003J!CTY|10YCY-1001A0003J,/home/ray/Dispa-SET_Unleash/RawData/Availabilt...,/home/ray/Dispa-SET_Unleash/RawData/Availabilt...,/home/ray/Dispa-SET_Unleash/RawData/Availabilt...,/home/ray/Dispa-SET_Unleash/RawData/Availabilt...,/home/ray/Dispa-SET_Unleash/RawData/Availabilt...
5,Czech Republic,CZ,https://transparency.entsoe.eu/generation/r2/a...,CET+(UTC+1)+/+CEST+(UTC+2),2023,UTC,WET_WEST,CET_CEST,EET_EST,https://transparency.entsoe.eu/generation/r2/a...,https://transparency.entsoe.eu/generation/r2/a...,https://transparency.entsoe.eu/generation/r2/a...,https://transparency.entsoe.eu/generation/r2/a...,CTY|10YCZ-CEPS-----N!CTY|10YCZ-CEPS-----N,/home/ray/Dispa-SET_Unleash/RawData/Availabilt...,/home/ray/Dispa-SET_Unleash/RawData/Availabilt...,/home/ray/Dispa-SET_Un

<div style="text-align: left; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman;">
5.4. Country Daily Data Download Links
</div>

<div style="text-align: left; margin-left: 0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
Creating all download links for daily data by each European country.
<br>
This links are formed using the base url links values of the standard_time_data_file previosuly created for each time zone (UTC, CET/CEST, WET/WEST, EET/EEST).
<br>
365 links are created for each country and saved in a .csv file which name is formed by the acronim of the country (AT, BE, DE, etc) and the suffix "_UTC", "_CET_CEST", "_WET_WEST", "_EET_EEST" correspondingly. 
<br>
These files are storaged in the folder which the same country code (AT, BE, DE, etc) in the path: /Local/Path/Dispa-SET_Unleash/RawData/AvailabiltyFactors.
<br>
Each one of these links are going to be used to download the tables that contains the daily aggregated power generation and conmsumption by technology type.
<br>
If it is true that the ENTSO E web page lets download a yearly file by country at once, the same does not content important values of the compsumtion disagregated by source. However this unavailable Data is obtained using the proceeding described in this script.
</div>

In [46]:
df = pd.read_csv(standard_time_data_file_path)

# Function to generate URLs for each day of the year
def generate_urls(base_url, year):
    urls = []
    # Iterate over each day of the year
    for day_of_year in range(1, 366 if year % 4 == 0 else 365):
        # Construct the date string in the format dd.mm.yyyy
        date_str = datetime.date(year, 1, 1) + datetime.timedelta(day_of_year - 1)
        date_str = date_str.strftime("%d.%m.%Y")
        # Replace the date part in the base URL
        url = base_url.replace("01.01.2023", date_str)  # Replace the year part if needed
        urls.append(url)
    return urls

# Specify the year
year = 2023

# Iterate over each row of the DataFrame
for index, row in df.iterrows():
    # Iterate over each pair of URL base column and destination file path column
    for base_col, file_path_col in [("UTC_Url_Country_Base", "UTC_Country_File_Path"),
                                    ("WET_WEST_Url_Country_Base", "WET_WEST_Country_File_Path"),
                                    ("CET_CEST_Url_Country_Base", "CET_CEST_Country_File_Path"),
                                    ("EET_EEST_Url_Country_Base", "EET_EEST_Country_File_Path")]:
        base_url = row[base_col]
        country_file_path = row[file_path_col]
        # Generate URLs for the current row
        urls = generate_urls(base_url, year)
        # Save the generated URLs to the file specified in the corresponding destination file path column
        with open(country_file_path, "w") as file:
            for url in urls:
                file.write(url + "\n")
        print(f"Generated URLs for {base_col} and saved to {file_path_col}")
print("All URLs generated and saved successfully!")

Generated URLs for UTC_Url_Country_Base and saved to UTC_Country_File_Path
Generated URLs for WET_WEST_Url_Country_Base and saved to WET_WEST_Country_File_Path
Generated URLs for CET_CEST_Url_Country_Base and saved to CET_CEST_Country_File_Path
Generated URLs for EET_EEST_Url_Country_Base and saved to EET_EEST_Country_File_Path
Generated URLs for UTC_Url_Country_Base and saved to UTC_Country_File_Path
Generated URLs for WET_WEST_Url_Country_Base and saved to WET_WEST_Country_File_Path
Generated URLs for CET_CEST_Url_Country_Base and saved to CET_CEST_Country_File_Path
Generated URLs for EET_EEST_Url_Country_Base and saved to EET_EEST_Country_File_Path
Generated URLs for UTC_Url_Country_Base and saved to UTC_Country_File_Path
Generated URLs for WET_WEST_Url_Country_Base and saved to WET_WEST_Country_File_Path
Generated URLs for CET_CEST_Url_Country_Base and saved to CET_CEST_Country_File_Path
Generated URLs for EET_EEST_Url_Country_Base and saved to EET_EEST_Country_File_Path
Generated 

<div style="text-align: left; margin-left: 3.0em; font-weight: bold; font-size: 17px; font-family: TimesNewRoman;">
5.5. UTC Raw Data File
</div>